In [122]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTENC

df = pd.read_excel(r"C:\Users\ashwi\GUVI_Projects\Job\Tensaw\Ass\AR - performance review - input.xlsx",header=2)

In [123]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df['payment_amount'] = df['payment_amount'].replace('[\$,]', '', regex=True).astype(float)
df['balance'] = df['balance'].replace('[\$,]', '', regex=True).astype(float)
df['denied'] = df['denial_reason'].notnull().astype(int)
df['payment_amount'].fillna(df['payment_amount'].median(), inplace=True)
df['balance'].fillna(df['balance'].median(), inplace=True)
for col in ['insurance_company', 'physician_name', 'cpt_code']:
    df[col].fillna(df[col].mode()[0], inplace=True)

<>:3: SyntaxWarning: invalid escape sequence '\$'
<>:4: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
<>:4: SyntaxWarning: invalid escape sequence '\$'
C:\Users\ashwi\AppData\Local\Temp\ipykernel_21500\3263926301.py:3: SyntaxWarning: invalid escape sequence '\$'
  df['payment_amount'] = df['payment_amount'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\ashwi\AppData\Local\Temp\ipykernel_21500\3263926301.py:4: SyntaxWarning: invalid escape sequence '\$'
  df['balance'] = df['balance'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\ashwi\AppData\Local\Temp\ipykernel_21500\3263926301.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

In [124]:
le_insurance = LabelEncoder()
le_physician = LabelEncoder()
le_cpt = LabelEncoder()
df['insurance_company'] = le_insurance.fit_transform(df['insurance_company'])
df['physician_name'] = le_physician.fit_transform(df['physician_name'])
df['cpt_code'] = le_cpt.fit_transform(df['cpt_code'])

X = df[['insurance_company', 'physician_name', 'cpt_code', 'payment_amount', 'balance']]
y = df['denied']
cat_indices = [0, 1, 2]

smote_nc = SMOTENC(categorical_features=cat_indices, random_state=42)
X_resampled, y_resampled = smote_nc.fit_resample(X, y)

In [125]:
##Recreation of datapoints

In [126]:
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['denied'] = y_resampled

df_resampled['payment_amount'] = (df_resampled['payment_amount'] / 10).round() * 10
df_resampled['balance'] = (df_resampled['balance'] / 10).round() * 10
df_resampled['payment_amount'] = df_resampled['payment_amount'].astype(int)
df_resampled['balance'] = df_resampled['balance'].astype(int)

df_resampled['insurance_company'] = le_insurance.inverse_transform(df_resampled['insurance_company'].astype(int))
df_resampled['physician_name'] = le_physician.inverse_transform(df_resampled['physician_name'].astype(int))
df_resampled['cpt_code'] = le_cpt.inverse_transform(df_resampled['cpt_code'].astype(int))
denial_reasons = ['16 - Missing information', '45 - Charge exceeds fee schedule', '96 - Non-covered service', '']
df_resampled['denied'] = df_resampled['denied'].fillna(0).astype(int)
df_resampled['denial_reason'] = "Not Denied"
denied_mask = df_resampled['denied'] == 1
df_resampled.loc[denied_mask, 'denial_reason'] = np.random.choice(
    denial_reasons, size=denied_mask.sum(), replace=True
)
df_resampled['denial_reason'] = df_resampled['denial_reason'].replace('', 'Not Denied').fillna("Not Denied")

In [127]:
df_final = df_resampled[['cpt_code', 'insurance_company', 'physician_name', 'payment_amount', 'balance', 'denial_reason']]
df_final = df_final.sample(n=500, replace=True, random_state=42)
output_path = r'C:\Users\ashwi\GUVI_Projects\Job\Tensaw\Ass\AR_performance_review_synthetic.xlsx'
df_final.to_excel(output_path, index=False)
print("Generated your dataset")

Generated your dataset
